In [61]:
# Preparations
import pandas as pd

# Define links
file_path = "tlozovskyi-flight-delay/data/"
file = "flights.csv"
flights_file = f"{file_path}{file}"

# Load the CSV file
df = pd.read_csv(flights_file)

# Replace all null values with zero
df_filled = df.fillna(0)

In [62]:
# Verify that there are no more nulls
print(df.isnull().sum())

# Display the first few rows of the cleansed data
df_filled.head()

# # Check schema
pd.set_option('display.max_columns', None)  # Show all columns

# # Show column names and data types
print(df.dtypes)

df_filled.head(2)

Year                    0
Month                   0
DayofMonth              0
DayOfWeek               0
Carrier                 0
OriginAirportID         0
OriginAirportName       0
OriginCity              0
OriginState             0
DestAirportID           0
DestAirportName         0
DestCity                0
DestState               0
CRSDepTime              0
DepDelay                0
DepDel15             2761
CRSArrTime              0
ArrDelay                0
ArrDel15                0
Cancelled               0
dtype: int64
Year                   int64
Month                  int64
DayofMonth             int64
DayOfWeek              int64
Carrier               object
OriginAirportID        int64
OriginAirportName     object
OriginCity            object
OriginState           object
DestAirportID          int64
DestAirportName       object
DestCity              object
DestState             object
CRSDepTime             int64
DepDelay               int64
DepDel15             float64
CRS

,Year,Month,DayofMonth,DayOfWeek,Carrier,OriginAirportID,OriginAirportName,OriginCity,OriginState,DestAirportID,DestAirportName,DestCity,DestState,CRSDepTime,DepDelay,DepDel15,CRSArrTime,ArrDelay,ArrDel15,Cancelled
0,2013,9,16,1,DL,15304,Tampa International,Tampa,FL,12478,John F. Kennedy International,New York,NY,1539,4,0.0,1824,13,0,0
1,2013,9,23,1,WN,14122,Pittsburgh International,Pittsburgh,PA,13232,Chicago Midway International,Chicago,IL,710,3,0.0,740,22,1,0


In [56]:
# Percentage of delayed flights by day and airport
file_name = "delay_chance_by_day_and_airport.csv"
# Group by DayOfWeek and DestAirportID, then calculate the mean of ArrDel15 (1 if delayed > 15 min, 0 otherwise)
delay_chance = (
    df_filled.groupby(['DayOfWeek', 'DestAirportID'])['ArrDel15']
    .mean()
    .reset_index()
    .rename(columns={'ArrDel15': 'delay_chance'})
    .sort_values('delay_chance', ascending=False)
)
# Save to CSV
delay_chance.to_csv(f"{file_path}{file_name}", index=False)

delay_chance.head()

,DayOfWeek,DestAirportID,delay_chance
229,4,11618,0.409524
240,4,12953,0.374468
327,5,14027,0.348485
237,4,12478,0.347924
256,4,13931,0.344828


In [57]:
# Extract unique destination airport IDs and names, and save to a CSV file
file_name = "airports.csv"

# Select relevant columns and drop duplicates
airports = df_filled[['DestAirportID', 'DestAirportName']].drop_duplicates().sort_values('DestAirportID', ascending=True)

# Save to CSV
airports.to_csv(f"{file_path}{file_name}", index=False)

airports.head()

,DestAirportID,DestAirportName
155,10140,Albuquerque International Sunport
381,10299,Ted Stevens Anchorage International
4,10397,Hartsfield-Jackson Atlanta International
45,10423,Austin - Bergstrom International
249,10529,Bradley International


In [58]:
# Percentage of delayed flights by airline
file_name = "airline_delays.csv"

airline_delays = (
    df_filled.groupby('Carrier')['ArrDel15']
    .mean()
    .reset_index()
    .rename(columns={'ArrDel15': 'delay_chance'})
    .sort_values('delay_chance', ascending=False)
)
# Save to CSV
airline_delays.to_csv(f"{file_path}{file_name}", index=False)

airline_delays.head()

,Carrier,delay_chance
9,MQ,0.312679
5,EV,0.269168
3,B6,0.251413
6,F9,0.242227
14,WN,0.236498


In [59]:
# Most delayed origin-destination airport pairs
file_name = "most_delayed_routes.csv"

route_delays = (
    df_filled.groupby(['OriginAirportID', 'DestAirportID'])['ArrDel15']
    .mean()
    .reset_index()
    .rename(columns={'ArrDel15': 'delay_chance'})
    .sort_values('delay_chance', ascending=False)
)
# Save to CSV
route_delays.to_csv(f"{file_path}{file_name}", index=False)

route_delays.head()

,OriginAirportID,DestAirportID,delay_chance
1568,13495,11278,1.000000
1951,14122,14492,1.000000
363,11042,14027,1.000000
2062,14635,14307,1.000000
1082,12478,14524,0.727273


In [63]:
# Best on-time performance by destination city
file_name = "best_performers.csv"

city_ontime = (
    df_filled.groupby('DestCity')['ArrDel15']
    .mean()
    .reset_index()
    .rename(columns={'ArrDel15': 'delay_chance'})
    .sort_values('delay_chance')
)
# Save to CSV
city_ontime.to_csv(f"{file_path}{file_name}", index=False)
city_ontime.head()

,DestCity,delay_chance
27,Long Beach,0.120048
24,Kahului,0.131363
20,Honolulu,0.144592
1,Anchorage,0.144772
53,Salt Lake City,0.152015


In [49]:
# Average delay chance by scheduled departure hour
df_filled['DepHour'] = df_filled['CRSDepTime'] // 100  # Extract hour from time
hourly_delays = (
    df_filled.groupby('DepHour')['ArrDel15']
    .mean()
    .reset_index()
    .rename(columns={'ArrDel15': 'delay_chance'})
    .sort_values('DepHour')
)
hourly_delays.head()

,DepHour,delay_chance
0,0,0.187621
1,1,0.110638
2,2,0.180000
3,3,0.272727
4,4,0.266667
